In [1]:
# https://greeksharifa.github.io/machine_learning/2019/12/21/FM/#reference

In [2]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import os,sys,inspect
import gc
from tqdm import tqdm
import random

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

from load import *
from evals import *

import warnings
warnings.filterwarnings('ignore')

In [3]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import optimizers, callbacks, layers, losses
from tensorflow.keras.layers import Dense, Concatenate, Activation, Add, BatchNormalization, Dropout, Input, Embedding, Flatten, Multiply
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)
        
def mish(x):
    return x*tf.math.tanh(tf.math.softplus(x))

def leakyrelu(x, factor=0.2):
    return tf.maximum(x, factor*x)

## Load

In [4]:
df = load_data('../data/ml-100k/u.data', threshold=3)
uuid = df['userId'].unique()
uiid = df['movieId'].unique()


In [5]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.15, random_state=SEED, stratify=df['userId'].values)

In [6]:
tr_X = np.hstack([to_categorical(train['userId'], len(uuid)), to_categorical(train['movieId'], len(uiid))])
test_X = np.hstack([to_categorical(test['userId'], len(uuid)), to_categorical(test['movieId'], len(uiid))])

## Model

In [7]:
# v1
class FM_layer(keras.Model):
    def __init__(self, x_dim, latent_dim, w_reg=1e-4, v_reg=1e-4):
        super(FM_layer, self).__init__()
        self.x_dim = x_dim
        self.latent_dim = latent_dim
        
        self.w_reg = w_reg
        self.v_reg = v_reg

    def build(self, input_shape):
        self.w_0 = self.add_weight(shape=(1, ),
                                  initializer=tf.zeros_initializer(),
                                  trainable=True)
        
        self.w = self.add_weight(shape=(self.x_dim, 1), 
                             initializer=tf.random_normal_initializer(),
                                trainable=True,
                                regularizer=l2(self.w_reg))
        
        self.V = self.add_weight(shape=(self.x_dim, self.latent_dim), 
                             initializer=tf.random_normal_initializer(),
                                trainable=True,
                                regularizer=l2(self.v_reg))

    def call(self, inputs):
        linear_terms = tf.reduce_sum(tf.matmul(inputs, self.w), axis=1)

        interactions = 0.5 * tf.reduce_sum(
            tf.pow(tf.matmul(inputs, self.V), 2)
            - tf.matmul(tf.pow(inputs, 2), tf.pow(self.V, 2)),
            1,
            keepdims=False
        )

        y_hat = (self.w_0 + linear_terms + interactions)

        return y_hat
    
# v2
# class FM_layer(keras.Model):
#     def __init__(self, x_dim, latent_dim):
#         super(FM_layer, self).__init__()
#         self.x_dim = x_dim
#         self.latent_dim = latent_dim

#         self.w_0 = tf.Variable([0.], shape=(1, ))
#         self.w = tf.Variable(tf.random.normal((self.x_dim, 1), 0, 0.5), 
#                              shape=(self.x_dim, 1))
#         self.V = tf.Variable(tf.random.normal((self.x_dim, self.latent_dim), 0, 0.5), 
#                              shape=(self.x_dim, self.latent_dim))

#     def call(self, inputs):
#         linear_terms = tf.reduce_sum(tf.matmul(inputs, self.w), axis=1)

#         interactions = 0.5 * tf.reduce_sum(
#             tf.pow(tf.matmul(inputs, self.V), 2)
#             - tf.matmul(tf.pow(inputs, 2), tf.pow(self.V, 2)),
#             1,
#             keepdims=False
#         )

#         y_hat = (self.w_0 + linear_terms + interactions)
        
#         return y_hat

## Train

In [8]:
class FM(tf.keras.Model):
    def __init__(self, x_dim, latnt_dim):
        super(FM, self).__init__()
        self.fm = FM_layer(x_dim, latnt_dim)

    def call(self, inputs):
        fm_outputs = self.fm(inputs)
        outputs = tf.nn.sigmoid(fm_outputs)
        return outputs

In [9]:
fm = FM(tr_X.shape[1], 8)

In [10]:
fm.compile(loss='binary_crossentropy', optimizer='adam')
fm.fit(tr_X, train['rating'].values,
      epochs=10,
      shuffle=True,
      validation_split=0.1)

Epoch 1/10
2391/2391 [==============================] - 6s 2ms/step - loss: 0.6640 - val_loss: 0.6465
Epoch 2/10
2391/2391 [==============================] - 6s 2ms/step - loss: 0.6325 - val_loss: 0.6301
Epoch 3/10
2391/2391 [==============================] - 6s 2ms/step - loss: 0.6196 - val_loss: 0.6242
Epoch 4/10
2391/2391 [==============================] - 6s 2ms/step - loss: 0.6143 - val_loss: 0.6218
Epoch 5/10
2391/2391 [==============================] - 6s 2ms/step - loss: 0.6119 - val_loss: 0.6210
Epoch 6/10
2391/2391 [==============================] - 6s 2ms/step - loss: 0.6107 - val_loss: 0.6206
Epoch 7/10
2391/2391 [==============================] - 6s 2ms/step - loss: 0.6101 - val_loss: 0.6206
Epoch 8/10
2391/2391 [==============================] - 6s 2ms/step - loss: 0.6098 - val_loss: 0.6206
Epoch 9/10
2391/2391 [==============================] - 6s 2ms/step - loss: 0.6097 - val_loss: 0.6206
Epoch 10/10
2391/2391 [==============================] - 6s 2ms/step - loss: 0.609

## Eval

In [11]:
pred = fm.predict(test_X)
# np.mean(np.square(test['rating'].values, pred.flatten()))


In [12]:
# accuracy
np.sum(np.where(pred>0.5, 1, 0).flatten() == test['rating'].values) / len(pred)

0.7033333333333334

In [13]:
from sklearn.metrics import precision_score, recall_score,  roc_auc_score, precision_recall_fscore_support

print(roc_auc_score(test['rating'].values, pred))
print(precision_score(test['rating'].values, np.where(pred>0.5, 1, 0)))
print(recall_score(test['rating'].values, np.where(pred>0.5, 1, 0)))

0.7663188323983955
0.6991350906095551
0.8163019956720365
